# CPABE Encryption

In [7]:
import hashlib
import os
import random

# Function to generate random medical attributes
def generate_random_attributes():
    roles = ['Doctor', 'Nurse', 'Surgeon', 'Therapist', 'Pharmacist']
    departments = ['Cardiology', 'Neurology', 'Orthopedics', 'Pediatrics', 'Oncology']
    return [random.choice(roles), random.choice(departments)]

# Function to generate a user key
def generate_user_key(user_attributes, master_key):
    attr_string = '_'.join(sorted(user_attributes)).encode('utf-8')
    return hashlib.sha256(attr_string + master_key).digest()

# Generate master key (in a real-world scenario, this should be stored securely)
master_key = os.urandom(32)

# Generate random attributes for a user
attributes = generate_random_attributes()
user_id = '12345'  # Example user ID, replace with actual user ID
user_key = generate_user_key(attributes, master_key)

# Save the key to a file
key_folder = 'keys'
os.makedirs(key_folder, exist_ok=True)
key_file_path = os.path.join(key_folder, f'user_key_{user_id}.bin')

with open(key_file_path, 'wb') as key_file:
    key_file.write(user_key)

print(f"Secret Key for User ID {user_id} with attributes {attributes} saved to {key_file_path}")


Secret Key for User ID 12345 with attributes ['Nurse', 'Neurology'] saved to keys\user_key_12345.bin


In [9]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import os

def load_user_key(user_id):
    key_file_path = os.path.join('keys', f'user_key_{user_id}.bin')
    with open(key_file_path, 'rb') as key_file:
        return key_file.read()

def encrypt_file(file_path, key):
    with open(file_path, 'rb') as f:
        data = f.read()

    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv

    encrypted_file_path = file_path + '.enc'
    with open(encrypted_file_path, 'wb') as ef:
        ef.write(iv + ct_bytes)

    return encrypted_file_path

# User ID and file to encrypt
user_id = '12345'  # Replace with the target user's ID
file_to_encrypt = 'C:/Users/ROHAN/IotSimulation/CPABE/ex1.txt.bin'

# Load user key and encrypt file
user_key = load_user_key(user_id)
encrypted_file_path = encrypt_file(file_to_encrypt, user_key)

print(f"File {file_to_encrypt} encrypted to {encrypted_file_path}")


File C:/Users/ROHAN/IotSimulation/CPABE/ex1.txt.bin encrypted to C:/Users/ROHAN/IotSimulation/CPABE/ex1.txt.bin.enc


In [16]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import os

def load_user_key(user_id):
    key_file_path = os.path.join('keys', f'user_key_{user_id}.bin')
    with open(key_file_path, 'rb') as key_file:
        return key_file.read()

def decrypt_file(encrypted_file_path, key, output_folder):
    with open(encrypted_file_path, 'rb') as ef:
        iv = ef.read(16)  # AES block size is 16 bytes
        ct_bytes = ef.read()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct_bytes), AES.block_size)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Prepare the path for the decrypted file
    decrypted_file_name = os.path.basename(encrypted_file_path).replace('.enc', '')
    decrypted_file_path = os.path.join(output_folder, decrypted_file_name)

    with open(decrypted_file_path, 'wb') as df:
        df.write(pt)

    return decrypted_file_path

# User ID and file to decrypt
user_id = '12345'  # Replace with the target user's ID
file_to_decrypt = 'C:/Users/ROHAN/IotSimulation/CPABE/ex1.txt.bin.enc'  # The encrypted file
output_folder = 'decrypted_files'  # Folder to save decrypted files

# Load user key and decrypt file
user_key = load_user_key(user_id)
decrypted_file_path = decrypt_file(file_to_decrypt, user_key, output_folder)

print(f"File {file_to_decrypt} decrypted to {decrypted_file_path}")


File C:/Users/ROHAN/IotSimulation/CPABE/ex1.txt.bin.enc decrypted to decrypted_files\ex1.txt.bin


# Creating SK for the users and CSV to add to The User IoT data

In [1]:
import hashlib
import os
import csv

# Provided DataUser dictionary
DataUser = {
    'Doctor1': 'EMP001', 'Nurse1': 'EMP002', 'Surgeon1': 'EMP003', 'Therapist1': 'EMP004',
    'Pharmacist1': 'EMP005', 'Radiologist1': 'EMP006', 'Pathologist1': 'EMP007',
    'Nurse2': 'EMP008', 'Doctor2': 'EMP009', 'Surgeon2': 'EMP010', 'Therapist2': 'EMP011',
    'Pharmacist2': 'EMP012', 'Radiologist2': 'EMP013', 'Pathologist2': 'EMP014',
    'Nurse3': 'EMP015', 'Doctor3': 'EMP016', 'Surgeon3': 'EMP017', 'Therapist3': 'EMP018',
    'Pharmacist3': 'EMP019', 'Radiologist3': 'EMP020'
}

# Function to generate a secret key for a user
def generate_user_key(user_role, master_key, user_id):
    role_string = user_role.encode('utf-8') + user_id.encode('utf-8')
    return hashlib.sha256(role_string + master_key).digest()

# Generate a master key (in a real-world scenario, this should be stored securely)
master_key = os.urandom(32)

# Directory to store secret keys
key_folder = 'employee_keys'
os.makedirs(key_folder, exist_ok=True)

# File to store user data
csv_file_path = 'DataUser.csv'

# Prepare to write data to CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['UserID', 'Role'])  # Writing header

    # Generate and store keys for the users in DataUser
    for user_role, user_id in DataUser.items():
        user_key = generate_user_key(user_role, master_key, user_id)

        # Save the key to a file
        key_file_path = os.path.join(key_folder, f'{user_id}_key.bin')
        with open(key_file_path, 'wb') as key_file:
            key_file.write(user_key)

        # Write user data to CSV file
        csv_writer.writerow([user_id, user_role])

        print(f"Generated key for {user_id} ({user_role}) stored in {key_file_path}")

print(f"User data saved to {csv_file_path}")


Generated key for EMP001 (Doctor1) stored in employee_keys\EMP001_key.bin
Generated key for EMP002 (Nurse1) stored in employee_keys\EMP002_key.bin
Generated key for EMP003 (Surgeon1) stored in employee_keys\EMP003_key.bin
Generated key for EMP004 (Therapist1) stored in employee_keys\EMP004_key.bin
Generated key for EMP005 (Pharmacist1) stored in employee_keys\EMP005_key.bin
Generated key for EMP006 (Radiologist1) stored in employee_keys\EMP006_key.bin
Generated key for EMP007 (Pathologist1) stored in employee_keys\EMP007_key.bin
Generated key for EMP008 (Nurse2) stored in employee_keys\EMP008_key.bin
Generated key for EMP009 (Doctor2) stored in employee_keys\EMP009_key.bin
Generated key for EMP010 (Surgeon2) stored in employee_keys\EMP010_key.bin
Generated key for EMP011 (Therapist2) stored in employee_keys\EMP011_key.bin
Generated key for EMP012 (Pharmacist2) stored in employee_keys\EMP012_key.bin
Generated key for EMP013 (Radiologist2) stored in employee_keys\EMP013_key.bin
Generated

# CPABE encryption

In [4]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import os
import hashlib
import re

def load_user_key(user_id, key_folder='employee_keys'):
    key_file_path = os.path.join(key_folder, f'{user_id}_key.bin')
    with open(key_file_path, 'rb') as key_file:
        return key_file.read()

def combine_keys(user_ids):
    combined_key = b''
    for user_id in user_ids:
        key = load_user_key(user_id)
        if key:
            combined_key += key
    return hashlib.sha256(combined_key).digest()

def extract_user_ids_from_filename(filename):
    pattern = r'EMP\d+'
    return re.findall(pattern, filename)

def encrypt_file(file_path, combined_key, output_folder):
    with open(file_path, 'rb') as f:
        data = f.read()

    cipher = AES.new(combined_key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv

    encrypted_file_path = os.path.join(output_folder, os.path.basename(file_path) + '.enc')
    with open(encrypted_file_path, 'wb') as ef:
        ef.write(iv + ct_bytes)

    return encrypted_file_path

# Directory containing files to encrypt
files_directory = 'C:/Users/ROHAN/IotSimulation/CPABE'
encrypted_files_directory = 'CPABE_Enc'
os.makedirs(encrypted_files_directory, exist_ok=True)

# Encrypt each file in the directory
for file in os.listdir(files_directory):
    file_path = os.path.join(files_directory, file)
    if os.path.isfile(file_path):
        user_ids = extract_user_ids_from_filename(file)
        combined_key = combine_keys(user_ids)
        encrypted_file_path = encrypt_file(file_path, combined_key, encrypted_files_directory)
        print(f"File {file_path} encrypted to {encrypted_file_path}")


File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_10_auth_EMP010_EMP008_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_10_auth_EMP010_EMP008_merged_1.txt.bin.enc
File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_11_auth_EMP007_EMP004_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_11_auth_EMP007_EMP004_merged_1.txt.bin.enc
File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_12_auth_EMP018_EMP012_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_12_auth_EMP018_EMP012_merged_1.txt.bin.enc
File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_13_auth_EMP020_EMP003_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_13_auth_EMP020_EMP003_merged_1.txt.bin.enc
File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_14_auth_EMP001_EMP009_EMP002_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_14_auth_EMP001_EMP009_EMP002_merged_1.txt.bin.enc
File C:/Users/ROHAN/IotSimulation/CPABE\key_iv_user_15_auth_EMP009_EMP013_EMP001_merged_1.txt.bin encrypted to CPABE_Enc\key_iv_user_15_auth_

In [6]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import os

def decrypt_file(encrypted_file_path, combined_key, output_folder):
    with open(encrypted_file_path, 'rb') as ef:
        iv = ef.read(16)
        ct_bytes = ef.read()

    cipher = AES.new(combined_key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct_bytes), AES.block_size)

    decrypted_file_path = os.path.join(output_folder, os.path.basename(encrypted_file_path).replace('.enc', ''))
    with open(decrypted_file_path, 'wb') as df:
        df.write(pt)

    return decrypted_file_path

# User IDs and encrypted file to decrypt
user_ids = ['EMP001', 'EMP002', 'EMP003']
encrypted_file_to_decrypt = 'C:/Users/ROHAN/IotSimulation/CPABE_Enc/key_iv_user_1_auth_EMP019_EMP017_merged_1.txt.bin.enc'

# Combine keys of specified users
combined_key = combine_keys(user_ids)

# Decrypt the file and save in 'decrypted_files' directory
decrypted_files_directory = 'CPABE_Dec'
os.makedirs(decrypted_files_directory, exist_ok=True)
decrypted_file_path = decrypt_file(encrypted_file_to_decrypt, combined_key, decrypted_files_directory)

print(f"File {encrypted_file_to_decrypt} decrypted to {decrypted_file_path}")


ValueError: Padding is incorrect.

In [26]:
# Predefined roles
roles = [
    'Doctor', 'Nurse', 'Surgeon', 'Therapist', 'Pharmacist',
    'Radiologist', 'Pathologist', 'Nurse', 'Doctor', 'Surgeon',
    'Therapist', 'Pharmacist', 'Radiologist', 'Pathologist', 'Nurse',
    'Doctor', 'Surgeon', 'Therapist', 'Pharmacist', 'Radiologist'
]

# Dictionary to store numbered roles and their associated employee IDs
employee_data = {}

# Counters for each role
role_counters = {role: 0 for role in set(roles)}

# Assign roles to employee IDs
for i, role in enumerate(roles, start=1):
    role_counters[role] += 1
    role_key = f'{role}{role_counters[role]}'
    employee_id = f'EMP{i:03d}'
    employee_data[role_key] = employee_id

# Output the structured employee data
print("Structured Employee Data:")
for role_key, employee_id in employee_data.items():
    print(f"'{role_key}': '{employee_id}'")


Structured Employee Data:
'Doctor1': 'EMP001'
'Nurse1': 'EMP002'
'Surgeon1': 'EMP003'
'Therapist1': 'EMP004'
'Pharmacist1': 'EMP005'
'Radiologist1': 'EMP006'
'Pathologist1': 'EMP007'
'Nurse2': 'EMP008'
'Doctor2': 'EMP009'
'Surgeon2': 'EMP010'
'Therapist2': 'EMP011'
'Pharmacist2': 'EMP012'
'Radiologist2': 'EMP013'
'Pathologist2': 'EMP014'
'Nurse3': 'EMP015'
'Doctor3': 'EMP016'
'Surgeon3': 'EMP017'
'Therapist3': 'EMP018'
'Pharmacist3': 'EMP019'
'Radiologist3': 'EMP020'


In [1]:
import os
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend

# Define your DataUser dictionary
DataUser = {
    'Doctor1': 'EMP001', 'Nurse1': 'EMP002', 'Surgeon1': 'EMP003', 'Therapist1': 'EMP004',
    'Pharmacist1': 'EMP005', 'Radiologist1': 'EMP006', 'Pathologist1': 'EMP007',
    'Nurse2': 'EMP008', 'Doctor2': 'EMP009', 'Surgeon2': 'EMP010', 'Therapist2': 'EMP011',
    'Pharmacist2': 'EMP012', 'Radiologist2': 'EMP013', 'Pathologist2': 'EMP014',
    'Nurse3': 'EMP015', 'Doctor3': 'EMP016', 'Surgeon3': 'EMP017', 'Therapist3': 'EMP018',
    'Pharmacist3': 'EMP019', 'Radiologist3': 'EMP020'
}


# Directory to save keys
keys_directory = "C:/Users/ROHAN/IotSimulation/CPABE-Keys"
os.makedirs(keys_directory, exist_ok=True)

def generate_keys_for_user(user_id):
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    # Generate public key
    public_key = private_key.public_key()

    # Save private key
    private_key_file = os.path.join(keys_directory, f"{user_id}_private_key.pem")
    with open(private_key_file, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # Save public key
    public_key_file = os.path.join(keys_directory, f"{user_id}_public_key.pem")
    with open(public_key_file, "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

# Generate and save keys for each user
for user, user_id in DataUser.items():
    generate_keys_for_user(user_id)
